# Import

In [1]:
import csv
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from tqdm import tqdm

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
tqdm.pandas()

# Read data

In [3]:
newsgroups_df = pd.read_csv("../../../Preprocessing/data/20News/master.csv", index_col=0)

In [4]:
with open("../../../Preprocessing/data/20News/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [5]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [18]:
n_class = len(class_labels[0])

In [20]:
model_nums = 2
vector_model_num = 0

In [19]:
def getGMM(vectors, n_class, seed, path):
    gmm = GaussianMixture(
        n_components=n_class, covariance_type="full", random_state=seed
    )
    gmm.fit(vectors)
    pickle.dump(gmm, open(path, "wb"))
    pred = gmm.predict(vectors)
    return pred

In [21]:
vectors_path = "../../data/20News/doc2vec/vector/"
models_path = "../../data/20News/doc2vec/GMM/model/"
pred_path = "../../data/20News/doc2vec/GMM/pred/"
for vector_size in tqdm(vector_sizes):
    for model_num in range(model_nums):
        vector_path = f"{vectors_path}{vector_size}/normalized/{vector_model_num}.csv"
        vectors = np.loadtxt(vector_path, delimiter=",")
        
        os.makedirs(f"{models_path}{vector_size}", exist_ok=True)
        pred = getGMM(vectors,n_class=n_class, seed=model_num, path=f"{models_path}{vector_size}/{model_num}.sav")
        
        # save prediction
        os.makedirs(f"{pred_path}{vector_size}", exist_ok=True)
        with open(f"{pred_path}{vector_size}/{model_num}.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerow(pred)

100%|██████████| 14/14 [4:03:01<00:00, 1041.54s/it]
